In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [19]:
summer_events = pd.read_csv('../Project-1/summer.csv')
winter_events = pd.read_csv('../Project-1/winter.csv')
dictionary_data = pd.read_csv('dictionary.csv', usecols=['Code', 'Country'])

country_mapping = dict(zip(dictionary_data['Code'], dictionary_data['Country']))

known_country_names =  {
    'URS': 'USSR-Prior 1992',
    'TCH': 'Czechoslovakia- Prior 1992',
    'ROU': 'Romania',
    'GDR': 'East Germany-Prior 1992',
    'EUN': 'Europe Unified Team',
    'FRG': 'Germany- Prior 1990',
    'EUA': 'Austria- pre AUT',
    'ZZX': 'Unknown',
    'ANZ': 'Australia-New Zealand',
    'BOH': 'Bohemia',
    'RU1': 'Russia',
    'YUG': 'Yugoslavia',
}

summer_events['Country Name'] = summer_events['Country'].map(country_mapping)


In [20]:
for index, row in summer_events.iterrows():
    country_code = row['Country']
    if pd.isna(row['Country Name']):
        if country_code in country_mapping:
            summer_events.at[index, 'Country Name'] = country_mapping[country_code]
        elif country_code in known_country_names:
            summer_events.at[index, 'Country Name'] = known_country_names[country_code]

In [21]:
track_field = summer_events.loc[(summer_events['Sport'] == 'Athletics') & (summer_events['Year'] >= 1948)]
track_field_df = pd.DataFrame(track_field)
track_field_df = track_field_df.drop(['Year','Country'],axis=1)
track_field_df = track_field_df.reindex(columns=['Country Name','Athlete','Gender','Event','Medal'])
track_field_df

,Country Name,Athlete,Gender,Event,Medal
7295,Sweden,"ALBERTSSON, Bertil",Men,10000M,Bronze
7296,Czechoslovakia- Prior 1992,"ZATOPEK, Emil",Men,10000M,Gold
7297,France,"MIMOUN, Alain",Men,10000M,Silver
7298,Switzerland,"SCHWAB, Erich Arthur Fritz",Men,10000M Walk,Bronze
7299,Sweden,"MIKAELSSON, John",Men,10000M Walk,Gold
...,...,...,...,...,...
29767,United States,"CLAYE, Will",Men,Triple Jump,Silver
29768,Italy,"DONATO, Fabrizio",Men,Triple Jump,Bronze
29769,Kazakhstan,"RYPAKOVA, Olga",Women,Triple Jump,Gold
29770,Colombia,"IBARGUEN, Caterine",Women,Triple Jump,Silver


In [15]:
track_field_df = track_field_df.drop(track_field_df[track_field_df['Event'] == '4X100M Relay'].index)
track_field_df = track_field_df.drop(track_field_df[track_field_df['Event'] == '4X400M Relay'].index)
track_field_df


,Country Name,Athlete,Gender,Event,Medal
7295,Sweden,"ALBERTSSON, Bertil",Men,10000M,Bronze
7296,Czechoslovakia- Prior 1992,"ZATOPEK, Emil",Men,10000M,Gold
7297,France,"MIMOUN, Alain",Men,10000M,Silver
7298,Switzerland,"SCHWAB, Erich Arthur Fritz",Men,10000M Walk,Bronze
7299,Sweden,"MIKAELSSON, John",Men,10000M Walk,Gold
...,...,...,...,...,...
29767,United States,"CLAYE, Will",Men,Triple Jump,Silver
29768,Italy,"DONATO, Fabrizio",Men,Triple Jump,Bronze
29769,Kazakhstan,"RYPAKOVA, Olga",Women,Triple Jump,Gold
29770,Colombia,"IBARGUEN, Caterine",Women,Triple Jump,Silver


In [17]:
track_field_df.to_csv('../Project-1/track_field.csv',index=False)
track_field_df = pd.read_csv('../Project-1/track_field.csv')